# Pydeck Earth Engine Introduction

This is an introduction to using [Pydeck](https://pydeck.gl) and [Deck.gl](https://deck.gl) with [Google Earth Engine](https://earthengine.google.com/) in Jupyter Notebooks.

If you wish to run this locally, you'll need to install some dependencies. Installing into a new Conda environment is recommended. To create and enter the environment, run:
```
conda create -n pydeck-ee -c conda-forge python jupyter notebook pydeck earthengine-api requests -y
source activate pydeck-ee
jupyter nbextension install --sys-prefix --symlink --overwrite --py pydeck
jupyter nbextension enable --sys-prefix --py pydeck
```
then open Jupyter Notebook with `jupyter notebook`.

Now in a Python Jupyter Notebook, let's first import required packages:

In [1]:
from pydeck_earthengine_layers import EarthEngineLayer
import pydeck as pdk
import requests
import ee

## Authentication

Using Earth Engine requires authentication. If you don't have a Google account approved for use with Earth Engine, you'll need to request access. For more information and to sign up, go to https://signup.earthengine.google.com/.

If you haven't used Earth Engine in Python before, you'll need to run the following authentication command. If you've previously authenticated in Python or the command line, you can skip the next line.

Note that this creates a prompt which waits for user input. If you don't see a prompt, you may need to authenticate on the command line with `earthengine authenticate` and then return here, skipping the Python authentication.

In [ ]:
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

## Create Map

Next it's time to create a map. Here we create an `ee.Image` object

In [ ]:
# Initialize objects
ee_layers = []
view_state = pdk.ViewState(latitude=37.7749295, longitude=-122.4194155, zoom=10, bearing=0, pitch=45)

In [4]:
# %%
# Add Earth Engine dataset
waterSurface = ee.Image('JRC/GSW1_0/GlobalSurfaceWater')
waterChange = waterSurface.select('transition')
 # Select Permanent Water Only:
Permanent_Water = 1 # value 1 represents pixels of permenant water, no change
waterMask = waterChange.eq(Permanent_Water) # Water mask boolean = 1 to detect whater bodies
# Map.setCenter(24.43874, 61.58173, 10)
# Map.addLayer(waterMask, {}, 'Water Mask')
# Map.centerObject(masked)
OnlyLakes = waterMask.updateMask(waterMask)

roi = ee.Geometry.Polygon(
        [[[22.049560546875, 61.171214253920965],
          [22.0330810546875, 60.833021871926185],
          [22.57415771484375, 60.83168327936567],
          [22.5714111328125, 61.171214253920965]]])

classes = OnlyLakes.reduceToVectors(**{
  'reducer': ee.Reducer.countEvery(),
  'geometry': roi,
  'scale': 30,
  'maxPixels': 1e10
})
simpleClasses = classes.geometry().simplify(50)

ee_layers.append(EarthEngineLayer(ee_object=ee.Image().paint(classes,0,2), vis_params={'palette':'red'}))
ee_layers.append(EarthEngineLayer(ee_object=ee.Image().paint(simpleClasses,0,2), vis_params={'palette':'blue'}))




Then just pass these layers to a `pydeck.Deck` instance, and call `.show()` to create a map:

In [8]:
r = pdk.Deck(layers=ee_layers, initial_view_state=view_state)
r.show()

DeckGLWidget(custom_libraries=[{'libraryName': 'EarthEngineLayerLibrary', 'resourceUri': 'https://cdn.jsdelivr…